# ML Model with Historical Features for Flight Delay Prediction

Uses ML algorithms with historical statistics as features to predict at booking time.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
import pickle
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [ ]:
raw_df = pd.read_csv("../files/flights.csv", low_memory=False)
df = raw_df[raw_df.ARRIVAL_DELAY.notna()].copy()

print(f"Total flights: {len(df):,}")
print(f"Columns: {df.columns.tolist()}")

## Calculate Historical Statistics

These will be features the ML model can use at prediction time.

In [ ]:
# Calculate historical averages by route/airline/month
route_airline_stats = df.groupby(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'MONTH'])['ARRIVAL_DELAY'].agg([
    ('hist_avg_delay', 'mean'),
    ('hist_std_delay', 'std'),
    ('hist_count', 'count')
]).reset_index()

print(f"\nRoute/Airline/Month combinations: {len(route_airline_stats):,}")

# Calculate by airline
airline_stats = df.groupby('AIRLINE')['ARRIVAL_DELAY'].agg([
    ('airline_avg_delay', 'mean')
]).reset_index()

# Calculate by route (any airline)
route_stats = df.groupby(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'MONTH'])['ARRIVAL_DELAY'].agg([
    ('route_avg_delay', 'mean')
]).reset_index()

# Calculate by origin airport + month
origin_stats = df.groupby(['ORIGIN_AIRPORT', 'MONTH'])['ARRIVAL_DELAY'].agg([
    ('origin_avg_delay', 'mean')
]).reset_index()

# Calculate by destination airport + month  
dest_stats = df.groupby(['DESTINATION_AIRPORT', 'MONTH'])['ARRIVAL_DELAY'].agg([
    ('dest_avg_delay', 'mean')
]).reset_index()

print(f"Airline stats: {len(airline_stats)}")
print(f"Route stats: {len(route_stats):,}")
print(f"Origin airport stats: {len(origin_stats):,}")
print(f"Destination airport stats: {len(dest_stats):,}")

## Merge Historical Features with Data

This simulates what we'd do at prediction time: look up historical stats.

In [ ]:
# Select columns
cols = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'MONTH', 'DAY_OF_WEEK', 
        'SCHEDULED_DEPARTURE', 'DISTANCE', 'ARRIVAL_DELAY']

available_cols = [col for col in cols if col in df.columns]
df_model = df[available_cols].copy()

# Merge historical stats
df_model = df_model.merge(route_airline_stats, 
                          on=['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'MONTH'], 
                          how='left')
df_model = df_model.merge(airline_stats, on='AIRLINE', how='left')
df_model = df_model.merge(route_stats, 
                          on=['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'MONTH'], 
                          how='left')
df_model = df_model.merge(origin_stats, on=['ORIGIN_AIRPORT', 'MONTH'], how='left')
df_model = df_model.merge(dest_stats, on=['DESTINATION_AIRPORT', 'MONTH'], how='left')

print(f"\nData with historical features: {df_model.shape}")
print(f"New columns: {[c for c in df_model.columns if 'hist_' in c or '_avg_' in c or '_std_' in c]}")

## Feature Engineering

In [ ]:
# Time-based features
if 'SCHEDULED_DEPARTURE' in df_model.columns:
    df_model['DEPARTURE_HOUR'] = df_model['SCHEDULED_DEPARTURE'] // 100
    df_model['IS_MORNING'] = (df_model['DEPARTURE_HOUR'] < 12).astype(int)
    df_model['IS_EVENING'] = (df_model['DEPARTURE_HOUR'] >= 18).astype(int)

if 'DAY_OF_WEEK' in df_model.columns:
    df_model['IS_WEEKEND'] = (df_model['DAY_OF_WEEK'] >= 6).astype(int)

if 'MONTH' in df_model.columns:
    df_model['IS_SUMMER'] = df_model['MONTH'].isin([6, 7, 8]).astype(int)
    df_model['IS_HOLIDAY'] = df_model['MONTH'].isin([11, 12]).astype(int)

if 'DISTANCE' in df_model.columns:
    df_model['IS_SHORT'] = (df_model['DISTANCE'] < 500).astype(int)
    df_model['IS_LONG'] = (df_model['DISTANCE'] > 2000).astype(int)

# Fill NaN in historical stats with overall means
for col in ['hist_avg_delay', 'hist_std_delay', 'airline_avg_delay', 
            'route_avg_delay', 'origin_avg_delay', 'dest_avg_delay']:
    if col in df_model.columns:
        df_model[col] = df_model[col].fillna(df_model['ARRIVAL_DELAY'].mean())

if 'hist_count' in df_model.columns:
    df_model['hist_count'] = df_model['hist_count'].fillna(0)

print(f"\nFeature engineering complete")
print(f"Total columns: {len(df_model.columns)}")

## Sample and Split

In [ ]:
# Sample 10k
sample_size = min(10000, len(df_model))
df_sample = df_model.sample(n=sample_size, random_state=42)

print(f"Sample size: {len(df_sample):,}")

# Drop columns not needed for prediction
drop_cols = ['ARRIVAL_DELAY', 'SCHEDULED_DEPARTURE']
X = df_sample.drop(drop_cols, axis=1, errors='ignore')
y = df_sample['ARRIVAL_DELAY']

# Identify categorical vs numeric
categorical_features = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
numeric_features = [col for col in X.columns if col not in categorical_features]

print(f"\nCategorical features: {categorical_features}")
print(f"Numeric features ({len(numeric_features)}): {numeric_features}")

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"\nTrain: {len(X_train):,} | Test: {len(X_test):,}")

## Build Models with LabelEncoder for Categories

XGBoost/LightGBM handle categorical features natively.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical features (handle unknown values)
encoders = {}
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

for col in categorical_features:
    encoders[col] = LabelEncoder()
    # Fit on all unique values from both train and test
    all_values = pd.concat([X_train[col], X_test[col]]).unique()
    encoders[col].fit(all_values.astype(str))
    X_train_encoded[col] = encoders[col].transform(X_train[col].astype(str))
    X_test_encoded[col] = encoders[col].transform(X_test[col].astype(str))

print("✓ Categorical features encoded")

## Train Models

In [ ]:
models = {
    'XGBoost': xgb.XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.3,
        reg_lambda=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=0
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        num_leaves=40,
        min_child_samples=15,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.3,
        reg_lambda=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    ),
    'Random Forest': RandomForestRegressor(
        n_estimators=200,
        max_depth=12,
        min_samples_split=15,
        min_samples_leaf=8,
        max_features=0.5,
        random_state=42,
        n_jobs=-1
    )
}

print("Training models...\n")
results = []

for name, model in models.items():
    print(f"[{name}]")
    print("  Training...", end=" ")
    model.fit(X_train_encoded, y_train)
    print("✓")
    
    print("  Cross-validating...", end=" ")
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_encoded, y_train, cv=kfold, scoring='r2', n_jobs=-1)
    print("✓")
    
    y_pred_train = model.predict(X_train_encoded)
    y_pred_test = model.predict(X_test_encoded)
    
    train_r2 = r2_score(y_train, y_pred_train) * 100
    test_r2 = r2_score(y_test, y_pred_test) * 100
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    results.append({
        'Model': name,
        'CV_R2': cv_scores.mean() * 100,
        'CV_Std': cv_scores.std() * 100,
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'Test_MAE': test_mae,
        'Test_RMSE': test_rmse,
        'Overfitting': train_r2 - test_r2,
        'predictions': y_pred_test,
        'model_obj': model
    })
    print(f"  Test R²: {test_r2:.2f}% | MAE: {test_mae:.2f}\n")

print("✓ All models trained")

## Results

In [ ]:
results_df = pd.DataFrame(results).sort_values('Test_R2', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON")
print("="*80)
print("\n{:<20} {:>10} {:>10} {:>10} {:>12}".format(
    "Model", "CV R²", "Test R²", "Test MAE", "Overfitting"
))
print("-" * 70)

for _, row in results_df.iterrows():
    print("{:<20} {:>9.2f}% {:>9.2f}% {:>9.2f} {:>11.2f}%".format(
        row['Model'], row['CV_R2'], row['Test_R2'], row['Test_MAE'], row['Overfitting']
    ))

best = results_df.iloc[0]
print("\n" + "="*80)
print(f"BEST MODEL: {best['Model']}")
print("="*80)
print(f"Test R²:  {best['Test_R2']:.2f}%")
print(f"Test MAE: {best['Test_MAE']:.2f} minutes")
print(f"Test RMSE: {best['Test_RMSE']:.2f} minutes")
print(f"CV R²: {best['CV_R2']:.2f}% ± {best['CV_Std']:.2f}%")
print(f"Overfitting Gap: {best['Overfitting']:.2f}%")

## Visualizations

In [ ]:
# Model comparison
fig = go.Figure()

fig.add_trace(go.Bar(
    name='CV R²',
    x=results_df['Model'],
    y=results_df['CV_R2'],
    error_y=dict(type='data', array=results_df['CV_Std'])
))

fig.add_trace(go.Bar(
    name='Test R²',
    x=results_df['Model'],
    y=results_df['Test_R2']
))

fig.update_layout(
    title='Model Comparison',
    xaxis_title='Model',
    yaxis_title='R² Score (%)',
    barmode='group'
)
fig.show()

In [ ]:
# Predicted vs Actual
best_predictions = best['predictions']

compare_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': best_predictions,
    'Error': np.abs(y_test - best_predictions)
})

fig = px.scatter(
    compare_df,
    x='Actual',
    y='Predicted',
    color='Error',
    title=f'{best["Model"]}: Predicted vs Actual',
    labels={'Actual': 'Actual Delay (min)', 'Predicted': 'Predicted Delay (min)'},
    color_continuous_scale='Reds',
    opacity=0.6
)

min_val = min(compare_df['Actual'].min(), compare_df['Predicted'].min())
max_val = max(compare_df['Actual'].max(), compare_df['Predicted'].max())

fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    name='Perfect',
    line=dict(dash='dash', color='black')
))

fig.show()

In [ ]:
# Error distribution
compare_df['Residual'] = compare_df['Actual'] - compare_df['Predicted']

fig = px.histogram(
    compare_df,
    x='Residual',
    nbins=50,
    title=f'{best["Model"]}: Error Distribution',
    labels={'Residual': 'Residual (min)'},
    marginal='box'
)
fig.show()

print(f"\nError Stats:")
print(f"Mean: {compare_df['Residual'].mean():.2f}")
print(f"Std: {compare_df['Residual'].std():.2f}")
print(f"Median Abs: {compare_df['Error'].median():.2f}")

In [ ]:
# Feature importance
if best['Model'] in ['XGBoost', 'LightGBM', 'Random Forest']:
    model_obj = best['model_obj']
    importances = model_obj.feature_importances_
    feature_names = X_train_encoded.columns.tolist()
    
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    print(f"\nTop 10 Features:")
    print(importance_df.head(10).to_string(index=False))
    
    fig = px.bar(
        importance_df.head(15),
        x='Importance',
        y='Feature',
        orientation='h',
        title=f'Feature Importance - {best["Model"]}'
    )
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

## Save Model

In [ ]:
# Save model and encoders
model_package = {
    'model': best['model_obj'],
    'encoders': encoders,
    'feature_names': X_train_encoded.columns.tolist(),
    'model_name': best['Model'],
    'test_r2': best['Test_R2'],
    'test_mae': best['Test_MAE']
}

with open('../modele_best.pkl', 'wb') as f:
    pickle.dump(model_package, f)

# Save historical stats for prediction time
stats_package = {
    'route_airline_stats': route_airline_stats,
    'airline_stats': airline_stats,
    'route_stats': route_stats,
    'origin_stats': origin_stats,
    'dest_stats': dest_stats
}

with open('../historical_stats.pkl', 'wb') as f:
    pickle.dump(stats_package, f)

print(f"✓ {best['Model']} saved")
print(f"✓ Test R²: {best['Test_R2']:.2f}%")
print(f"✓ Test MAE: {best['Test_MAE']:.2f} min")
print(f"✓ Historical stats saved for predictions")

## Summary

This ML model:
- Uses historical statistics as features
- Trains on full dataset with variance
- Works at booking time (no departure delay needed)
- Uses XGBoost/LightGBM/Random Forest
- Proper ML workflow for homework